<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/JARVIS_OpenMM_CoarseGraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://cg-openmm.readthedocs.io/en/latest/examples/replica_exchange_simulation.html

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [1]:
!conda --version

conda 23.11.0


In [2]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [3]:
!pip install -q -U git+https://github.com/shirtsgroup/cg_openmm

  Preparing metadata (setup.py) ... done


In [4]:
!pip install -q -U git+https://github.com/shirtsgroup/analyze_foldamers.git

  Preparing metadata (setup.py) ... done


In [5]:
!conda install mdtraj mpi4py numpy openmm openmmtools physical_validation -y --quiet

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mdtraj
    - mpi4py
    - numpy
    - openmm
    - openmmtools
    - physical_validation


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astunparse-1.6.3           |     pyhd8ed1ab_0          15 KB  conda-forge
    blosc-1.21.5               |       hc2324a3_1          48 KB  conda-forge
    brotli-1.1.0               |       hd590300_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hd590300_1          19 KB  conda-forge
    c-blosc2-2.14.4            |       hb4ffafa_1         329 KB  conda-forge
    ca-certificates-2024.6.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    cftime-1.6.3               |  py310h261611a

In [17]:
!wget https://raw.githubusercontent.com/shirtsgroup/cg_openmm/773a0c76ddd6d4192bfdd7c66affbcec86d5530b/examples/run_standard_MD_simulation/initial_structure_trial_1.pdb -O initial_structure_trial_1.pdb

--2024-06-05 01:42:40--  https://raw.githubusercontent.com/shirtsgroup/cg_openmm/773a0c76ddd6d4192bfdd7c66affbcec86d5530b/examples/run_standard_MD_simulation/initial_structure_trial_1.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4023 (3.9K) [text/plain]
Saving to: ‘initial_structure_trial_1.pdb’

initial_structure_t 100%[===================>]   3.93K  --.-KB/s    in 0s      

2024-06-05 01:42:41 (73.3 MB/s) - ‘initial_structure_trial_1.pdb’ saved [4023/4023]



In [18]:
import os
import pickle
import time

import cg_openmm
import numpy as np
from cg_openmm.cg_model.cgmodel import CGModel
from cg_openmm.simulation.tools import run_simulation
from openmm import unit
from openmm.app.pdbfile import PDBFile

# This example demonstrates how to run a standard (single temperature) NVT simulation
# for a coarse-grained model in OpenMM.

sim_begin = time.perf_counter()

#---------------------------#
# Simulation job parameters #
#---------------------------#

# Set output directory:
output_directory = "output"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# Replica exchange simulation settings:
total_simulation_time = 5.0 * unit.nanosecond
simulation_time_step = 5.0 * unit.femtosecond
total_steps = int(np.floor(total_simulation_time / simulation_time_step))
temperature = 300.0 * unit.kelvin
collision_frequency = 5 / unit.picosecond
print_frequency = 5000  # Number of steps to skip when printing output

include_bond_forces = True
include_bond_angle_forces = True
include_nonbonded_forces = True
include_torsion_forces = True
constrain_bonds = False

#--------------------------------------------#
# Particle definitions and oligomer topology #
#--------------------------------------------#

mass = 100.0 * unit.amu

# Mass and charge are defaults.
# Backbone particle:
bb = {
    "particle_type_name": "bb",
    "sigma": 2.25 * unit.angstrom,
    "epsilon": 1.50 * unit.kilojoules_per_mole,
    "mass": mass
}

# Sidechain particle:
sc = {
    "particle_type_name": "sc",
    "sigma": 3.50 * unit.angstrom,
    "epsilon": 5.00 * unit.kilojoules_per_mole,
    "mass": mass
}

# Monomer definition:
A = {
    "monomer_name": "A",
    "particle_sequence": [bb, sc],
    "bond_list": [[0, 1]],
    "start": 0,
    "end": 0,
}

# Residue sequence:
sequence = 24 * [A]

#--------------------------#
# Harmonic bond parameters #
#--------------------------#

# Bond definitions:
bond_lengths = {"default_bond_length": 2.44 * unit.angstrom}

bond_force_constants = {
    "default_bond_force_constant": 15000 * unit.kilojoule_per_mole / unit.nanometer / unit.nanometer
}

#---------------------------#
# Harmonic angle parameters #
#---------------------------#

# Bond angle definitions:
bond_angle_force_constants = {
    "default_bond_angle_force_constant": 150 * unit.kilojoule_per_mole / unit.radian / unit.radian
}

equil_bond_angles = {
    "default_equil_bond_angle": 127.5 * unit.degrees,
    "bb_bb_bb_equil_bond_angle": 105.5 * unit.degrees}

#-----------------------------#
# Periodic torsion parameters #
#-----------------------------#

# Torsion angle definitions:
torsion_force_constants = {
    "default_torsion_force_constant": 0.0 * unit.kilojoule_per_mole,
    "bb_bb_bb_bb_torsion_force_constant": 5 * unit.kilojoule_per_mole}

torsion_phase_angles = {
    "sc_bb_bb_sc_torsion_phase_angle": 0 * unit.degrees,
    "bb_bb_bb_bb_torsion_phase_angle": (16.7-180) * unit.degrees,
    "bb_bb_bb_sc_torsion_phase_angle": 0 * unit.degrees,
}

torsion_periodicities = {
    "default_torsion_periodicity": 1,
}

# Get initial positions from file:
pdb_path = "initial_structure_trial_1.pdb"
positions = PDBFile(pdb_path).getPositions()

# Build a coarse grained model:
cgmodel = CGModel(
    particle_type_list=[bb, sc],
    bond_lengths=bond_lengths,
    bond_force_constants=bond_force_constants,
    bond_angle_force_constants=bond_angle_force_constants,
    torsion_force_constants=torsion_force_constants,
    equil_bond_angles=equil_bond_angles,
    torsion_phase_angles=torsion_phase_angles,
    torsion_periodicities=torsion_periodicities,
    include_nonbonded_forces=include_nonbonded_forces,
    include_bond_forces=include_bond_forces,
    include_bond_angle_forces=include_bond_angle_forces,
    include_torsion_forces=include_torsion_forces,
    constrain_bonds=constrain_bonds,
    positions=positions,
    sequence=sequence,
    monomer_types=[A],
)

# Store the cg model so that we can do various analyses.
cgmodel.export("stored_cgmodel.pkl")

# Run the simulation
run_simulation(
    cgmodel,
    total_simulation_time,
    simulation_time_step,
    temperature,
    friction=collision_frequency,
    print_frequency=print_frequency,
    output_directory=output_directory,
)


sim_end = time.perf_counter()

print(f'simulation run time: {sim_end-sim_begin}')

charge not defined for particle type bb using default charge: 0.0 e
charge not defined for particle type sc using default charge: 0.0 e
No bond_force_constants definition provided for 'bb_sc_bond_force_constant', setting to 15000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_sc_bond_length', setting to 2.44 A
No bond_force_constants definition provided for 'bb_bb_bond_force_constant', setting to 15000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_bb_bond_length', setting to 2.44 A
No bond_angle_force_constants definition provided for 'sc_bb_bb_bond_angle_force_constant', setting to 150 kJ/(mol rad**2)
No equil_bond_angles definition provided for 'sc_bb_bb_equil_bond_angle', setting to 127.5 deg
No bond_angle_force_constants definition provided for 'bb_bb_bb_bond_angle_force_constant', setting to 150 kJ/(mol rad**2)
No torsion_force_constants definition provided for 'sc_bb_bb_sc_torsion_force_constant', setting to 0.0 kJ/mol
No torsion_periodicities definition 

In [28]:
#!~/anaconda3/bin/python

import os
import time

import numpy as np
import openmm
from cg_openmm.cg_model.cgmodel import CGModel
from cg_openmm.parameters.reweight import get_temperature_list
from cg_openmm.simulation.rep_exch import *
from openmm import unit
from openmmtools.cache import global_context_cache

# This example demonstrates how to run a OpenMM replica exchange simulation
# using a CGModel object.


rep_exch_begin = time.perf_counter()

#---------------------------------#
# Replica exchange job parameters #
#---------------------------------#

# Set output directory:
output_directory = "output"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
overwrite_files = True  # overwrite files.

# Use CUDA platform for GPU acceleration:
global_context_cache.platform = openmm.Platform.getPlatformByName("CUDA")

# Replica exchange simulation settings:
total_simulation_time = 50.0 * unit.nanosecond
simulation_time_step = 5.0 * unit.femtosecond
total_steps = int(np.floor(total_simulation_time / simulation_time_step))
output_data = os.path.join(output_directory, "output.nc")

number_replicas = 12
min_temp = 200.0 * unit.kelvin
max_temp = 600.0 * unit.kelvin
# Use logarithmic temperature spacing:
temperature_list = get_temperature_list(min_temp, max_temp, number_replicas)

exchange_frequency = 200 # Number of steps between exchange attempts
collision_frequency = 5 / unit.picosecond

include_bond_forces = True
include_bond_angle_forces = True
include_nonbonded_forces = True
include_torsion_forces = True
constrain_bonds = False

#--------------------------------------------#
# Particle definitions and oligomer topology #
#--------------------------------------------#

mass = 100.0 * unit.amu

# Mass and charge are defaults.
# Backbone particle:
bb = {
    "particle_type_name": "bb",
    "sigma": 2.25 * unit.angstrom,
    "epsilon": 1.50 * unit.kilojoules_per_mole,
    "mass": mass
}

# Sidechain particle:
sc = {
    "particle_type_name": "sc",
    "sigma": 3.50 * unit.angstrom,
    "epsilon": 5.00 * unit.kilojoules_per_mole,
    "mass": mass
}

# Monomer definition:
A = {
    "monomer_name": "A",
    "particle_sequence": [bb, sc],
    "bond_list": [[0, 1]],
    "start": 0,
    "end": 0,
}

# Residue sequence:
sequence = 24 * [A]

#--------------------------#
# Harmonic bond parameters #
#--------------------------#

# Bond definitions:
bond_lengths = {"default_bond_length": 2.44 * unit.angstrom}

bond_force_constants = {
    "default_bond_force_constant": 15000 * unit.kilojoule_per_mole / unit.nanometer / unit.nanometer
}

#---------------------------#
# Harmonic angle parameters #
#---------------------------#

# Bond angle definitions:
bond_angle_force_constants = {
    "default_bond_angle_force_constant": 150 * unit.kilojoule_per_mole / unit.radian / unit.radian
}

equil_bond_angles = {
    "default_equil_bond_angle": 127.5 * unit.degrees,
    "bb_bb_bb_equil_bond_angle": 105.5 * unit.degrees}

#-----------------------------#
# Periodic torsion parameters #
#-----------------------------#

# Torsion angle definitions:
torsion_force_constants = {
    "default_torsion_force_constant": 0.0 * unit.kilojoule_per_mole,
    "bb_bb_bb_bb_torsion_force_constant": 5 * unit.kilojoule_per_mole}

torsion_phase_angles = {
    "sc_bb_bb_sc_torsion_phase_angle": 0 * unit.degrees,
    "bb_bb_bb_bb_torsion_phase_angle": (16.7-180) * unit.degrees,
    "bb_bb_bb_sc_torsion_phase_angle": 0 * unit.degrees,
}

torsion_periodicities = {
    "default_torsion_periodicity": 1,
}

# Get initial positions from file:
pdb_path = "initial_structure_trial_1.pdb"
positions = PDBFile(pdb_path).getPositions()

# Build a coarse grained model:
cgmodel = CGModel(
    particle_type_list=[bb, sc],
    bond_lengths=bond_lengths,
    bond_force_constants=bond_force_constants,
    bond_angle_force_constants=bond_angle_force_constants,
    torsion_force_constants=torsion_force_constants,
    equil_bond_angles=equil_bond_angles,
    torsion_phase_angles=torsion_phase_angles,
    torsion_periodicities=torsion_periodicities,
    include_nonbonded_forces=include_nonbonded_forces,
    include_bond_forces=include_bond_forces,
    include_bond_angle_forces=include_bond_angle_forces,
    include_torsion_forces=include_torsion_forces,
    constrain_bonds=constrain_bonds,
    positions=positions,
    sequence=sequence,
    monomer_types=[A],
)

# Store the cg model so that we can do various analyses.
cgmodel.export("stored_cgmodel.pkl")

if not os.path.exists(output_data) or overwrite_files == True:
    run_replica_exchange(
        cgmodel.topology,
        cgmodel.system,
        cgmodel.positions,
        friction=collision_frequency,
        temperature_list=temperature_list,
        simulation_time_step=simulation_time_step,
        total_simulation_time=total_simulation_time,
        exchange_frequency=exchange_frequency,
        output_data=output_data,
    )
else:
    print("Replica output files exist")

rep_exch_end = time.perf_counter()

print(f'replica exchange run time: {rep_exch_end-rep_exch_begin}')

charge not defined for particle type bb using default charge: 0.0 e
charge not defined for particle type sc using default charge: 0.0 e


No bond_force_constants definition provided for 'bb_sc_bond_force_constant', setting to 15000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_sc_bond_length', setting to 2.44 A
No bond_force_constants definition provided for 'bb_bb_bond_force_constant', setting to 15000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_bb_bond_length', setting to 2.44 A
No bond_angle_force_constants definition provided for 'sc_bb_bb_bond_angle_force_constant', setting to 150 kJ/(mol rad**2)
No equil_bond_angles definition provided for 'sc_bb_bb_equil_bond_angle', setting to 127.5 deg
No bond_angle_force_constants definition provided for 'bb_bb_bb_bond_angle_force_constant', setting to 150 kJ/(mol rad**2)
No torsion_force_constants definition provided for 'sc_bb_bb_sc_torsion_force_constant', setting to 0.0 kJ/mol
No torsion_periodicities definition provided for 'sc_bb_bb_sc_torsion_periodicity', setting to 1
No torsion_force_constants definition provided for 'sc_bb_bb_bb_torsion_for

replica exchange run time: 5197.227624017


In [29]:


import os
import pickle

from cg_openmm.thermo.calc import *
from openmm import unit

# This example demonstrates how to calculate heat capacity as a function of temperature from
# replica exchange energies, with uncertainties estimated using pyMBAR.

# Note: process_replica_exchange_data should first be run to determine the determine the start
# of the production region and energy decorrelation time.


# Job settings
output_directory = 'output'
output_data = os.path.join(output_directory, "output.nc")

# Load in trajectory stats:
analysis_stats = pickle.load(open("analysis_stats_discard_20ns.pkl","rb"))

# Read the simulation coordinates for individual temperature replicas
C_v, dC_v, new_temperature_list, FWHM, Tm, Cv_height, N_eff = get_heat_capacity(
    output_data=output_data,
    frame_begin=analysis_stats["production_start"],
    sample_spacing=analysis_stats["energy_decorrelation"],
    num_intermediate_states=3,
    plot_file="heat_capacity.pdf",
)

print(f"T({new_temperature_list[0].unit})  Cv({C_v[0].unit})  dCv({dC_v[0].unit})")
for i, C in enumerate(C_v):
    print(f"{new_temperature_list[i]._value:>8.2f}{C_v[i]._value:>10.4f} {dC_v[i]._value:>10.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'analysis_stats_discard_20ns.pkl'

In [25]:
from simtk import unit
from cg_openmm.cg_model.cgmodel import CGModel

# Specify backbone (bb) and sidechain (sc) particle parameters:
sigma = 0.3 * unit.nanometer
epsilon = 2 * unit.kilojoule_per_mole
mass = 100 * unit.amu

bb = {"particle_type_name": "bb", "sigma": sigma, "epsilon": epsilon, "mass": mass}
sc = {"particle_type_name": "sc", "sigma": sigma, "epsilon": epsilon, "mass": mass}

# Define monomer (residue):
A = {
    "monomer_name": "A",
    "particle_sequence": [bb, sc],
    "bond_list": [[0, 1]],
    "start": 0,
    "end": 0}

# Specify bonded parameters:
bond_lengths = {
    "default_bond_length": 0.35 * unit.nanometer,
    "bb_bb_bb_bond_length": 0.40 * unit.nanometer}

bond_force_constants = {
    "default_bond_force_constant": 1000 * unit.kilojoule_per_mole / unit.nanometer**2}

equil_bond_angles = {
    "default_equil_bond_angle": 120.0 * unit.degrees,
    "bb_bb_bb_equil_bond_angle": 150.0 * unit.degrees}

bond_angle_force_constants = {
    "default_bond_angle_force_constant": 100.0 * unit.kilojoule_per_mole / unit.radian**2}

torsion_phase_angles = {
    "default_torsion_phase_angle": 150 * unit.degrees}

torsion_force_constants = {
    "default_torsion_force_constant": 2.0 * unit.kilojoule_per_mole,
    "bb_bb_bb_bb_torsion_force_constant": 5.0 * unit.kilojoule_per_mole}

torsion_periodicities = {
    "default_torsion_periodicity": 1}

# Define oligomer sequence:
sequence = 12 * [A]

# Initial particle positions determined from random builder

cgmodel = CGModel(
    particle_type_list=[bb, sc],
    bond_lengths=bond_lengths,
    bond_force_constants=bond_force_constants,
    bond_angle_force_constants=bond_angle_force_constants,
    torsion_force_constants=torsion_force_constants,
    equil_bond_angles=equil_bond_angles,
    torsion_phase_angles=torsion_phase_angles,
    torsion_periodicities=torsion_periodicities,
    include_nonbonded_forces=True,
    include_bond_forces=True,
    include_bond_angle_forces=True,
    include_torsion_forces=True,
    constrain_bonds=False,
    sequence=sequence,
    monomer_types=[A],
)

charge not defined for particle type bb using default charge: 0.0 e
charge not defined for particle type sc using default charge: 0.0 e
No bond_force_constants definition provided for 'bb_sc_bond_force_constant', setting to 1000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_sc_bond_length', setting to 0.35 nm
No bond_force_constants definition provided for 'bb_bb_bond_force_constant', setting to 1000 kJ/(nm**2 mol)
No bond_lengths definition provided for 'bb_bb_bond_length', setting to 0.35 nm
No bond_angle_force_constants definition provided for 'sc_bb_bb_bond_angle_force_constant', setting to 100.0 kJ/(mol rad**2)
No equil_bond_angles definition provided for 'sc_bb_bb_equil_bond_angle', setting to 120.0 deg
No bond_angle_force_constants definition provided for 'bb_bb_bb_bond_angle_force_constant', setting to 100.0 kJ/(mol rad**2)
No torsion_force_constants definition provided for 'sc_bb_bb_sc_torsion_force_constant', setting to 2.0 kJ/mol
No torsion_phase_angles definiti

In [15]:
cgmodel.bond_list

[[0, 1],
 [0, 2],
 [2, 3],
 [2, 4],
 [4, 5],
 [4, 6],
 [6, 7],
 [6, 8],
 [8, 9],
 [8, 10],
 [10, 11],
 [10, 12],
 [12, 13],
 [12, 14],
 [14, 15],
 [14, 16],
 [16, 17],
 [16, 18],
 [18, 19],
 [18, 20],
 [20, 21],
 [20, 22],
 [22, 23]]

In [16]:
cgmodel.topology.createTopology()

AttributeError: 'CGModel' object has no attribute 'topology'

In [26]:
cgmodel.export("stored_cgmodel.pkl")

In [27]:
import os

import numpy as np
import simtk.openmm as openmm
from cg_openmm.cg_model.cgmodel import CGModel
from cg_openmm.parameters.reweight import get_temperature_list
from cg_openmm.simulation.rep_exch import *
from openmmtools.cache import global_context_cache
from simtk import unit
import pickle

# Set output directory:
output_directory = "output"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# Use CUDA platform for GPU acceleration:
global_context_cache.platform = openmm.Platform.getPlatformByName("CUDA")

# Replica exchange simulation settings:
total_simulation_time = 50.0 * unit.nanosecond
simulation_time_step = 5.0 * unit.femtosecond
total_steps = int(np.floor(total_simulation_time / simulation_time_step))
output_data = os.path.join(output_directory, "output.nc")

number_replicas = 12
min_temp = 200.0 * unit.kelvin
max_temp = 600.0 * unit.kelvin
# Use logarithmic temperature spacing:
temperature_list = get_temperature_list(min_temp, max_temp, number_replicas)

exchange_frequency = 200 # Number of steps between exchange attempts
collision_frequency = 5 / unit.picosecond

# Load in a cgmodel:
cgmodel = pickle.load(open("stored_cgmodel.pkl","rb"))

run_replica_exchange(
    cgmodel.topology,
    cgmodel.system,
    cgmodel.positions,
    friction=collision_frequency,
    temperature_list=temperature_list,
    simulation_time_step=simulation_time_step,
    total_simulation_time=total_simulation_time,
    exchange_frequency=exchange_frequency,
    output_data=output_data,
)

AttributeError: 'CGModel' object has no attribute 'topology'

In [ ]:
cgmodel.

In [3]:
!git clone https://github.com/shirtsgroup/cg_openmm.git

Cloning into 'cg_openmm'...
remote: Enumerating objects: 17808, done.
remote: Counting objects: 100% (1697/1697), done.
remote: Compressing objects: 100% (565/565), done.
remote: Total 17808 (delta 1162), reused 1542 (delta 1115), pack-reused 16111
Receiving objects: 100% (17808/17808), 404.43 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (11305/11305), done.


In [4]:
import os
os.chdir("cg_openmm")

In [5]:
!python setup.py install

running install
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [6]:
os.chdir("/content")

In [7]:
!git clone https://github.com/shirtsgroup/analyze_foldamers.git

Cloning into 'analyze_foldamers'...
remote: Enumerating objects: 2292, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 2292 (delta 31), reused 33 (delta 21), pack-reused 2171
Receiving objects: 100% (2292/2292), 123.52 MiB | 23.67 MiB/s, done.
Resolving deltas: 100% (1265/1265), done.


In [8]:
os.chdir("analyze_foldamers")

In [9]:
!python setup.py install

running install
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [10]:
os.chdir("/content")

In [11]:
!conda install mdtraj mpi4py numpy openmm openmmtools physical_validation -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mdtraj
    - mpi4py
    - numpy
    - openmm
    - openmmtools
    - physical_validation


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    astunparse-1.6.3           |     pyhd8ed1ab_0          15 KB  conda-forge
    blosc-1.21.5               |       hc2324a3_1          48 KB  conda-forge
    brotli-1.1.0               |       hd590300_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hd590300_1          19 KB  conda-forge
    c-blosc2-2.14.4            |       hb4ffafa_1         329 KB  conda-forge
    ca

In [13]:
os.makedirs('Project')
os.chdir('Project')

In [26]:
!pip install  -U git+https://github.com/shirtsgroup/cg_openmm

  Cloning https://github.com/shirtsgroup/cg_openmm to /tmp/pip-req-build-sbpltr_s
  Running command git clone --filter=blob:none --quiet https://github.com/shirtsgroup/cg_openmm /tmp/pip-req-build-sbpltr_s
  Resolved https://github.com/shirtsgroup/cg_openmm to commit 773a0c76ddd6d4192bfdd7c66affbcec86d5530b
  Preparing metadata (setup.py) ... done


In [19]:
import numpy
numpy.__file__

'/usr/local/lib/python3.10/dist-packages/numpy/__init__.py'

In [25]:
!ls /usr/local/lib/python3.10/dist-packages/cg_openmm*

ls: cannot access '/usr/local/lib/python3.10/dist-packages/cg_openmm*': No such file or directory


In [28]:
!rm -rf /usr/local/lib/python3.10/dist-packages/cg_openmm*

In [29]:
from simtk import unit
from cg_openmm.cg_model.cgmodel import CGModel

# Specify backbone (bb) and sidechain (sc) particle parameters:
sigma = 0.3 * unit.nanometer
epsilon = 2 * unit.kilojoule_per_mole
mass = 100 * unit.amu

bb = {"particle_type_name": "bb", "sigma": sigma, "epsilon": epsilon, "mass": mass}
sc = {"particle_type_name": "sc", "sigma": sigma, "epsilon": epsilon, "mass": mass}

# Define monomer (residue):
A = {
    "monomer_name": "A",
    "particle_sequence": [bb, sc],
    "bond_list": [[0, 1]],
    "start": 0,
    "end": 0}

# Specify bonded parameters:
bond_lengths = {
    "default_bond_length": 0.35 * unit.nanometer,
    "bb_bb_bb_bond_length": 0.40 * unit.nanometer}

bond_force_constants = {
    "default_bond_force_constant": 1000 * unit.kilojoule_per_mole / unit.nanometer**2}

equil_bond_angles = {
    "default_equil_bond_angle": 120.0 * unit.degrees,
    "bb_bb_bb_equil_bond_angle": 150.0 * unit.degrees}

bond_angle_force_constants = {
    "default_bond_angle_force_constant": 100.0 * unit.kilojoule_per_mole / unit.radian**2}

torsion_phase_angles = {
    "default_torsion_phase_angle": 150 * unit.degrees}

torsion_force_constants = {
    "default_torsion_force_constant": 2.0 * unit.kilojoule_per_mole,
    "bb_bb_bb_bb_torsion_force_constant": 5.0 * unit.kilojoule_per_mole}

torsion_periodicities = {
    "default_torsion_periodicity": 1}

# Define oligomer sequence:
sequence = 12 * [A]

# Initial particle positions determined from random builder

cgmodel = CGModel(
    particle_type_list=[bb, sc],
    bond_lengths=bond_lengths,
    bond_force_constants=bond_force_constants,
    bond_angle_force_constants=bond_angle_force_constants,
    torsion_force_constants=torsion_force_constants,
    equil_bond_angles=equil_bond_angles,
    torsion_phase_angles=torsion_phase_angles,
    torsion_periodicities=torsion_periodicities,
    include_nonbonded_forces=True,
    include_bond_forces=True,
    include_bond_angle_forces=True,
    include_torsion_forces=True,
    constrain_bonds=False,
    sequence=sequence,
    monomer_types=[A],
)

ModuleNotFoundError: No module named 'cg_openmm.cg_model'